# LSTM time series forecasting

In [1]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("/content/MicrosoftStock.csv")
data.head()

,index,date,open,high,low,close,volume,Name
0,390198,2013-02-08,27.35,27.71,27.31,27.55,33318306,MSFT
1,390199,2013-02-11,27.65,27.92,27.50,27.86,32247549,MSFT
2,390200,2013-02-12,27.88,28.00,27.75,27.88,35990829,MSFT
3,390201,2013-02-13,27.93,28.11,27.88,28.03,41715530,MSFT
4,390202,2013-02-14,27.92,28.06,27.87,28.04,32663174,MSFT


In [3]:
data.shape

(1259, 8)

In [4]:
data.columns.tolist()

['index', 'date', 'open', 'high', 'low', 'close', 'volume', 'Name']

In [5]:
data.isnull().sum()

,0
index,0
date,0
open,0
high,0
low,0
close,0
volume,0
Name,0


In [6]:
data.duplicated().sum()

np.int64(0)